# Table of Contents
 <p>

In [38]:
import json
import hmac
import time
import base64
import hashlib
import requests
import pandas as pd
from requests.auth import AuthBase

In [41]:
PUBLIC = [
    'getmarkets', 
    'getcurrencies', 
    'getticker', 
    'getmarketsummaries', 
    'getmarketsummary', 
    'getorderbook', 
    'getmarkethistory'
]

MARKET = [
    'buylimit', 
    'buymarket', 
    'selllimit', 
    'sellmarket', 
    'cancel', 
    'getopenorders'
]

ACCOUNT = [
    'getbalances', 
    'getbalance', 
    'getdepositaddress', 
    'withdraw', 
    'getorder', 
    'getorderhistory', 
    'getwithdrawalhistory', 
    'getdeposithistory'
]

In [ ]:
class BittrexExchangeAuth(AuthBase):
    def __init__(self, api_key, secret_key):
        self.api_key = api_key
        self.secret_key = secret_key
        self.public = PUBLIC
        self.market = MARKET
        self.account = ACCOUNT
    
    def __call__(self, request):
        timestamp = str(time.time())
        message = timestamp + request.method + request.path_url + (request.body or '')
        hmac_key = base64.b64decode(self.secret_key)
        signature = hmac.new(hmac_key, message.encode('utf-8'), hashlib.sha256)
        digest = signature.digest()
        signature_b64 = base64.b64encode(digest).decode('utf-8').rstrip('\n')

        request.headers.update({
            'CB-ACCESS-SIGN': signature_b64,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-ACCESS-PASSPHRASE': self.passphrase,
            'Content-Type': 'application/json'
        })
        return request